In [73]:
import json
import csv, sys
import urllib.request

maxInt = sys.maxsize
decrement = True

while decrement:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    decrement = False
    try:
        csv.field_size_limit(maxInt)
    except OverflowError:
        maxInt = int(maxInt/10)
        decrement = True


file = open('binaryValues.csv', 'r')
reader = csv.reader(file)
data = []
for row in reader:
    data.append(row)

file.close()

In [74]:
from collections import defaultdict

def decode_binary_string(s):
    return ''.join(chr(int(s[i*8:i*8+8],2)) for i in range(len(s)//8))

In [75]:
decoded = decode_binary_string(data[1][0])

In [76]:
def getAssocVal(arr,jsonData):
    idx = 0
    val = jsonData.get(arr[idx])
    l = len(arr)
    while l > idx + 1:
        if arr[idx + 1].isnumeric():
            if len(val) >= int(arr[idx + 1]):
                val = val[int(arr[idx + 1])].get(arr[idx + 2])
            else:
                val = 0
            idx += 2
        else:
            val = val.get(arr[idx + 1])
            idx += 1
    return val

In [77]:
urlsToArr = defaultdict(list) 
urlsToJson = dict()

In [78]:
for s in decoded.split("https\x9e//")[1:]:
    url = s.split(",")[0]
    tempArr = s.split(",")[1]
    if urlsToArr.get(url,None) is None:
        urlsToArr[url] = []
    urlsToArr.get(url).append(tempArr)

In [85]:
list(urlsToArr.keys())

['data.cityofnewyork.us/api/views/kku6-nxdu',
 'data.kingcounty.gov/api/views/b27z-cdmk',
 'data.montgomerycountymd.gov/api/views/icn6-v9z3',
 'data.iowa.gov/api/views/s3p7-wy6w',
 'data.maryland.gov/api/views/f53i-bwcd',
 'data.baltimorecity.gov/api/views/xviu-ezkt',
 'data.ct.gov/api/views/kbxi-4ia7',
 'data.cdc.gov/api/views/hn4x-zwk7',
 'data.cdc.gov/api/views/mr8w-325u',
 'data.cityofnewyork.us/api/views/5t4n-d72c',
 'data.cityofnewyork.us/api/views/dpec-ucu7',
 'data.cdc.gov/api/views/e8kx-wbww',
 'data.austintexas.gov/api/views/spbg-9v94',
 'data.cdc.gov/api/views/3h58-x6cd',
 'data.sfgov.org/api/views/58un-vqfs',
 'data.kingcounty.gov/api/views/yaai-7frk',
 'data.ny.gov/api/views/dg63-4siq',
 'data.seattle.gov/api/views/3k2p-39jp',
 'data.mo.gov/api/views/wp2d-vk85',
 'data.medicare.gov/api/views/77hc-ibv8',
 'data.baltimorecity.gov/api/views/aivj-4x23',
 'data.ny.gov/api/views/6amx-2pbv',
 'data.cityofnewyork.us/api/views/f9bf-2cp4',
 'data.wa.gov/api/views/6ffb-b8kq']

In [80]:
for url in urlsToArr.keys():
    tUrl = 'https://' + url
    urlsToJson[tUrl] = urllib.request.urlopen(tUrl).read()

In [81]:
cn = 0.0
for k in urlsToJson.keys():
    for s in urlsToArr.keys():
        if k[8:] == s:
            for arr in urlsToArr.get(s):
                if arr.find('.') > 0 and 'count' in arr:
#                     print k , arr
                    t = getAssocVal(arr.split('.'),json.loads(urlsToJson.get(k)))
#                     print(t)          
                    cn += t

In [82]:
cn

535.0

In [83]:
f = open("results.csv","w")
f.write("Results\n")
f.write(str(cn))
f.close()